<a href="https://colab.research.google.com/github/widowswail-03/attribution-analysis-model/blob/main/attribution_analysis_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import datetime

In [7]:
df = pd.read_excel("sample_data.xlsx")
closing_price = pd.read_excel("closing_prices.xlsx")
closing_price['Date'] = pd.to_datetime(closing_price['Date']).dt.date.astype(str)
cashflow_df = pd.read_excel("cashflow_with_expense.xlsx")
target_date = "2025-04-03"

securities = df['Security'].unique()
securities_df = pd.DataFrame(securities, columns = ['Security'])
securities_df['quantity'] = 0
securities_df['daily_invested_amount'] = 0
securities_df['daily_returns'] = 0
securities_df['net_daily_returns'] = 0
securities_df['avg_entry_price'] = 0


In [8]:
def calculate_day_returns(df, securities_df, closing_price, cashflow_df, target_date):
    securities_df = securities_df.copy()

    # Initialize columns
    for col in ['quantity', 'daily_invested_amount', 'daily_returns', 'net_daily_returns', 'avg_entry_price']:
        if col not in securities_df.columns:
            securities_df[col] = 0.0

    # Ensure data types
    df['Quantity'] = df['Quantity'].astype(float)
    df['Settlement Amount'] = df['Settlement Amount'].astype(float)
    securities_df['quantity'] = securities_df['quantity'].astype(float)
    securities_df['avg_entry_price'] = securities_df['avg_entry_price'].astype(float)

    # Handle cashflow_df
    cashflow_df = cashflow_df.copy() if not cashflow_df.empty else pd.DataFrame(columns=['Date', 'Cash'])
    if 'Cash' in cashflow_df.columns:
        cashflow_df['Cash'] = cashflow_df['Cash'].astype(float)
    else:
        print(f"Warning: 'Cash' column not found in cashflow_df. Using 0 for cash flows.")
        cashflow_df['Cash'] = 0.0

    # Convert dates to string 'YYYY-MM-DD'
    df = df.copy()
    closing_price = closing_price.copy()
    df['Tran Date'] = pd.to_datetime(df['Tran Date']).dt.strftime('%Y-%m-%d')
    cashflow_df['Date'] = pd.to_datetime(cashflow_df['Date']).dt.strftime('%Y-%m-%d')

    # Debug closing_price dates
    print(f"closing_price['Date'] before conversion:\n", closing_price['Date'])
    print(f"closing_price['Date'] types:\n", closing_price['Date'].apply(type))

    # Convert closing_price dates to strings
    closing_price['Date'] = pd.to_datetime(closing_price['Date']).dt.date.astype(str)
    closing_price['Date'] = closing_price['Date'].str.strip()

    print(f"closing_price['Date'] after conversion:\n", closing_price['Date'])
    print(f"closing_price['Date'] values: {closing_price['Date'].tolist()}")

    # Filter transactions
    day_df = df[df['Tran Date'] == target_date].copy()
    print("Filtered day_df:\n", day_df)

    if day_df.empty:
        print(f"No transactions found for {target_date}")

    # Aggregate buy/sell
    buy_df = day_df[day_df['Transaction Description'].str.lower() == 'buy'].groupby('Security').agg({
        'Quantity': 'sum',
        'Settlement Amount': 'sum'
    }).rename(columns={'Quantity': 'buy_quantity', 'Settlement Amount': 'buy_amount'})

    if not buy_df.empty:
        buy_df['buy_avg_entry_price'] = buy_df['buy_amount'] / buy_df['buy_quantity'].replace(0, 1)
    else:
        buy_df = pd.DataFrame(columns=['buy_quantity', 'buy_amount', 'buy_avg_entry_price'], index=pd.Index([], name='Security'))

    sell_df = day_df[day_df['Transaction Description'].str.lower() == 'sell'].groupby('Security').agg({
        'Quantity': 'sum',
        'Settlement Amount': 'sum'
    }).rename(columns={'Quantity': 'sell_quantity', 'Settlement Amount': 'sell_amount'})

    if sell_df.empty:
        sell_df = pd.DataFrame(columns=['sell_quantity', 'sell_amount'], index=pd.Index([], name='Security'))

    # Merge buy/sell
    securities_df = securities_df.merge(buy_df, on='Security', how='left').merge(sell_df, on='Security', how='left')
    securities_df['buy_quantity'] = securities_df['buy_quantity'].fillna(0)
    securities_df['buy_amount'] = securities_df['buy_amount'].fillna(0)
    securities_df['sell_quantity'] = securities_df['sell_quantity'].fillna(0)
    securities_df['sell_amount'] = securities_df['sell_amount'].fillna(0)
    securities_df['avg_entry_price'] = securities_df['buy_avg_entry_price'].fillna(securities_df['avg_entry_price'])
    securities_df = securities_df.drop(columns=['buy_avg_entry_price'], errors='ignore')
    print("After merging buy/sell:\n", securities_df)

    # Update quantities
    securities_df['quantity'] += securities_df['buy_quantity'] - securities_df['sell_quantity']
    securities_df['daily_invested_amount'] += securities_df['buy_amount']
    securities_df['daily_returns'] += securities_df['sell_amount']
    print("After buy/sell updates:\n", securities_df)

    # Sort closing_price
    closing_price = closing_price.sort_values('Date').reset_index(drop=True)

    # Get target date index
    target_date_index = closing_price[closing_price['Date'] == target_date].index
    if len(target_date_index) == 0:
        print(f"No price data for {target_date}, using last available prices if possible")
    else:
        target_date_index = target_date_index[0]

    # Get cash balance
    cash_today_df = cashflow_df[cashflow_df['Date'] == target_date]
    cash_today = cash_today_df['Cash'].iloc[0] if not cash_today_df.empty else 0.0
    print(f"Cash balance on {target_date}: {cash_today}")

    # Get previous day's cash balance
    cash_yesterday = 0.0
    if target_date_index > 0:
        prev_date = closing_price.iloc[target_date_index - 1]['Date']
        cash_yesterday_df = cashflow_df[cashflow_df['Date'] == prev_date]
        cash_yesterday = cash_yesterday_df['Cash'].iloc[0] if not cash_yesterday_df.empty else 0.0
    elif len(closing_price) > 0:
        prev_date = closing_price['Date'].iloc[0]
        cash_yesterday_df = cashflow_df[cashflow_df['Date'] == prev_date]
        cash_yesterday = cash_yesterday_df['Cash'].iloc[0] if not cash_yesterday_df.empty else 0.0
    print(f"Cash balance on previous day: {cash_yesterday}")

    # Add cash row to securities_df
    cash_row = pd.DataFrame({
        'Security': ['Cash'],
        'quantity': [cash_today],
        'daily_invested_amount': [0.0],
        'daily_returns': [0.0],
        'net_daily_returns': [0.0],
        'avg_entry_price': [0.0],
        'buy_quantity': [0.0],
        'buy_amount': [0.0],
        'sell_quantity': [0.0],
        'sell_amount': [0.0]
    })
    securities_df = pd.concat([securities_df, cash_row], ignore_index=True)
    print("After adding Cash row:\n", securities_df)

    # Calculate returns using specified formula
    weights = []
    returns = []
    initial_values = []

    for security in securities_df['Security']:
        if security == 'Cash':
            continue
        if security not in closing_price.columns:
            print(f"No price column found for {security} on {target_date}, skipping")
            securities_df.loc[securities_df['Security'] == security, 'net_daily_returns'] = 0.0
            continue

        # Get today's price
        if target_date in closing_price['Date'].values:
            close_price_today = closing_price.loc[closing_price['Date'] == target_date, security].iloc[0]
        else:
            # Use last available price
            last_date = closing_price['Date'].max()
            if pd.to_datetime(last_date) < pd.to_datetime(target_date):
                close_price_today = closing_price.loc[closing_price['Date'] == last_date, security].iloc[0]
            else:
                print(f"No valid price for {security} on or before {target_date}, setting return to 0")
                securities_df.loc[securities_df['Security'] == security, 'net_daily_returns'] = 0.0
                continue

        quantity_today = securities_df.loc[securities_df['Security'] == security, 'quantity'].iloc[0]
        buy_quantity = securities_df.loc[securities_df['Security'] == security, 'buy_quantity'].iloc[0]
        sell_quantity = securities_df.loc[securities_df['Security'] == security, 'sell_quantity'].iloc[0]
        buy_amount = securities_df.loc[securities_df['Security'] == security, 'buy_amount'].iloc[0]
        sell_amount = securities_df.loc[securities_df['Security'] == security, 'sell_amount'].iloc[0]

        # Calculate quantity held yesterday
        quantity_yesterday = quantity_today - buy_quantity + sell_quantity

        # Get yesterday's price
        close_price_yesterday = 0.0
        if target_date_index > 0:
            prev_date = closing_price.iloc[target_date_index - 1]['Date']
            close_price_yesterday = closing_price.loc[closing_price['Date'] == prev_date, security].iloc[0]
        elif target_date_index == 0:
            # First day, no yesterday price, return = 0
            close_price_yesterday = 0.0

        # Calculate return
        holding_gain_loss = (quantity_today * close_price_today) - (quantity_yesterday * close_price_yesterday)
        numerator = sell_amount + holding_gain_loss - buy_amount
        denominator = quantity_yesterday * close_price_yesterday if quantity_yesterday != 0 else 0.0

        security_return = numerator / denominator if denominator != 0 else 0.0
        securities_df.loc[securities_df['Security'] == security, 'net_daily_returns'] = security_return

        # Store for portfolio return
        initial_value = quantity_yesterday * close_price_yesterday
        weights.append(initial_value)
        returns.append(security_return)
        initial_values.append(initial_value)

        print(f"{security}: sell_amount={sell_amount}, holding_gain_loss={holding_gain_loss}, buy_amount={buy_amount}, "
              f"quantity_yesterday={quantity_yesterday}, close_yesterday={close_price_yesterday}, return={security_return}")

    # Add cash
    weights.append(cash_yesterday)
    returns.append(0.0)
    initial_values.append(cash_yesterday)

    print("After calculating returns:\n", securities_df)

    # Calculate portfolio return
    total_initial_value = sum(initial_values)
    if total_initial_value != 0:
        weights = [w / total_initial_value for w in weights]
    else:
        weights = [0.0] * len(weights)

    total_return = sum(w * r for w, r in zip(weights, returns))
    print(f"Weights: {weights}")
    print(f"Individual Returns: {returns}")
    print(f"Total Portfolio Return: {total_return}")

    securities_df = securities_df.drop(columns=['buy_quantity', 'buy_amount', 'sell_quantity', 'sell_amount'], errors='ignore')

    return securities_df, total_return, cash_today

In [9]:
print("closing_price:\n", closing_price)
print("closing_price columns:", closing_price.columns.tolist())
print("closing_price dates:", closing_price['Date'].tolist())
print("closing_price date types:", closing_price['Date'].apply(type).tolist())
print("securities_df:\n", securities_df)
print("securities_df securities:", securities_df['Security'].tolist())
print("cashflow_df:\n", cashflow_df)
print("cashflow_df columns:", cashflow_df.columns.tolist())

# Define start and end dates
start_date = '2023-07-11'
end_date = '2023-08-11'

# Generate date range
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Initialize results
portfolio_results = []

# Loop to compute net value
for date in date_range:
    date_str = date.strftime('%Y-%m-%d')
    print(f"\nProcessing date: {date_str}")
    print(f"Checking if {date_str} in closing_price['Date']: {date_str in closing_price['Date'].values}")

    try:
        securities_df, total_return, cash_balance = calculate_day_returns(
            df, securities_df, closing_price, cashflow_df, date_str
        )

        # Calculate net value (securities only)
        net_value = 0.0
        closing_price['Date'] = closing_price['Date'].str.strip()
        if date_str in closing_price['Date'].values:
            close_prices = closing_price[closing_price['Date'] == date_str].iloc[0]
        else:
            # Use last available prices
            last_date = closing_price['Date'].max()
            if pd.to_datetime(last_date) < pd.to_datetime(date_str):
                close_prices = closing_price[closing_price['Date'] == last_date].iloc[0]
                print(f"Using last available prices from {last_date} for {date_str}")
            else:
                print(f"No prices available for {date_str}")
                close_prices = None

        if close_prices is not None:
            print(f"Close prices for {date_str}:\n", close_prices)
            non_cash_securities = securities_df[securities_df['Security'] != 'Cash']
            print(f"Non-cash securities for {date_str}:\n", non_cash_securities[['Security', 'quantity']])
            if non_cash_securities.empty:
                print(f"No non-cash securities found for {date_str}")
            for security in non_cash_securities['Security']:
                if security in closing_price.columns:
                    quantity = securities_df.loc[securities_df['Security'] == security, 'quantity'].iloc[0]
                    close_price = close_prices[security]
                    contribution = quantity * close_price
                    print(f"Security: {security}, Quantity: {quantity}, Close Price: {close_price}, Contribution: {contribution}")
                    net_value += contribution
                else:
                    print(f"No price column found for {security} on {date_str}")

        print(f"Calculated Net Value for {date_str}: {net_value}")
        portfolio_results.append({
            'Date': date_str,
            'Total_Return': total_return,
            'Net_Value': net_value
        })

        print(f"Total Return for {date_str}: {total_return}")
        print(f"Net Value for {date_str}: {net_value}")
        print(f"Updated securities_df:\n", securities_df)

    except ValueError as e:
        print(f"ValueError on {date_str}: {e}")
        cash_balance = cashflow_df[cashflow_df['Date'] == date_str]['Cash'].iloc[0] if not cashflow_df[cashflow_df['Date'] == date_str].empty else 0.0
        portfolio_results.append({
            'Date': date_str,
            'Total_Return': 0.0,
            'Net_Value': 0.0
        })
    except Exception as e:
        print(f"Unexpected error on {date_str}: {e}")
        cash_balance = cashflow_df[cashflow_df['Date'] == date_str]['Cash'].iloc[0] if not cashflow_df[cashflow_df['Date'] == date_str].empty else 0.0
        portfolio_results.append({
            'Date': date_str,
            'Total_Return': 0.0,
            'Net_Value': 0.0
        })

# Create results DataFrame
portfolio_results_df = pd.DataFrame(portfolio_results)

# Save to Excel
excel_file = 'portfolio_value.xlsx'
try:
    portfolio_results_df.to_excel(excel_file, index=False, engine='openpyxl')
    print(f"\nResults saved to '{excel_file}'")
except Exception as e:
    print(f"Error saving Excel file: {e}")

# Print results
print("\nPortfolio Results:\n", portfolio_results_df)

# Cumulative return
cumulative_return = (1 + portfolio_results_df['Total_Return']).prod() - 1
print(f"Cumulative Return: {cumulative_return}")

closing_price:
            Date  Security 1  Security 2  Security 3  Security 4  Security 5  \
0    2023-07-10  536.624390     3487.65     3546.70     3972.15     1893.30   
1    2023-07-11  534.835815     3514.05     3774.80     4017.35     1890.65   
2    2023-07-12  538.959534     3529.00     3800.50     4097.65     1901.40   
3    2023-07-13  533.146606     3511.15     3815.15     4129.85     1892.35   
4    2023-07-14  543.828430     3519.35     3874.35     4542.60     1918.40   
..          ...         ...         ...         ...         ...         ...   
656  2025-04-26  366.200012     5033.50     5507.00     4779.60     2861.10   
657  2025-04-27  366.200012     5033.50     5507.00     4779.60     2861.10   
658  2025-04-28  367.000000     5025.80     5598.00     4748.10     2920.60   
659  2025-04-29  365.799988     5030.80     5570.50     4881.10     2899.90   
660  2025-04-30  359.200012     5116.00     5520.00     4605.30     2895.40   

     Security 6  Security 7  Securi

/tmp/ipython-input-8-1869318090.py:70: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  securities_df['sell_quantity'] = securities_df['sell_quantity'].fillna(0)
/tmp/ipython-input-8-1869318090.py:71: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  securities_df['sell_amount'] = securities_df['sell_amount'].fillna(0)


Security: Security 39, Quantity: 0.0, Close Price: 606.2, Contribution: 0.0
Security: Security 40, Quantity: 0.0, Close Price: 0.0, Contribution: 0.0
Security: Security 41, Quantity: 0.0, Close Price: 277.55, Contribution: 0.0
Security: Security 42, Quantity: 0.0, Close Price: 0.0, Contribution: 0.0
Security: Security 43, Quantity: 0.0, Close Price: 0.0, Contribution: 0.0
Security: Security 44, Quantity: 0.0, Close Price: 0.0, Contribution: 0.0
Security: Security 45, Quantity: 0.0, Close Price: 0.0, Contribution: 0.0
Security: Security 46, Quantity: 0.0, Close Price: 0.0, Contribution: 0.0
Security: Security 47, Quantity: 0.0, Close Price: 483.3, Contribution: 0.0
Security: Security 48, Quantity: 0.0, Close Price: 693.0, Contribution: 0.0
No price column found for nan on 2023-07-11
Calculated Net Value for 2023-07-11: 107267548.64333495
Total Return for 2023-07-11: 0.0
Net Value for 2023-07-11: 107267548.64333495
Updated securities_df:
        Security     quantity  daily_invested_amou

/tmp/ipython-input-8-1869318090.py:70: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  securities_df['sell_quantity'] = securities_df['sell_quantity'].fillna(0)
/tmp/ipython-input-8-1869318090.py:71: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  securities_df['sell_amount'] = securities_df['sell_amount'].fillna(0)


       Security      quantity  daily_invested_amount  daily_returns  \
0    Security 1  3.601300e+04           1.941554e+07            0.0   
1    Security 2  3.315000e+03           1.167154e+07            0.0   
2    Security 3  4.121000e+03           1.547588e+07            0.0   
3    Security 4  2.855000e+03           1.166754e+07            0.0   
4    Security 5  6.134000e+03           1.170162e+07            0.0   
5    Security 6  1.078900e+04           1.946874e+07            0.0   
6    Security 7  3.162000e+03           1.533203e+07            0.0   
7    Security 8  3.212500e+04           1.903624e+07            0.0   
8    Security 9  1.626200e+04           1.167020e+07            0.0   
9   Security 10  2.001000e+04           1.896773e+07            0.0   
10  Security 11  1.126500e+04           1.553458e+07            0.0   
11  Security 12  1.744100e+04           1.550765e+07            0.0   
12  Security 13  4.204400e+04           1.539527e+07            0.0   
13  Se

...

Security 14: sell_amount=0.0, holding_gain_loss=1391598.3499999978, buy_amount=1259998.161376, quantity_yesterday=111641.0, close_yesterday=217.55, return=0.005418432982492751
Security 15: sell_amount=0.0, holding_gain_loss=1604732.0000000037, buy_amount=1082060.256984, quantity_yesterday=13138.0, close_yesterday=1473.6, return=0.02699728873146336
Security 16: sell_amount=0.0, holding_gain_loss=162801.19999999925, buy_amount=262706.088064, quantity_yesterday=8632.0, close_yesterday=2048.65, return=-0.005649467802289518
Security 17: sell_amount=0.0, holding_gain_loss=39494.919999999925, buy_amount=314729.289896, quantity_yesterday=44448.0, close_yesterday=67.75, return=-0.09139893639003348
Security 18: sell_amount=0.0, holding_gain_loss=2495154.3000000003, buy_amount=2438613.795618, quantity_yesterday=7055.0, close_yesterday=178.9, return=0.044797349565559356
Security 19: sell_amount=0.0, holding_gain_loss=354831.8999999985, buy_amount=385293.44061, quantity_yesterday=22888.0, close_yes

/tmp/ipython-input-8-1869318090.py:68: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  securities_df['buy_quantity'] = securities_df['buy_quantity'].fillna(0)
/tmp/ipython-input-8-1869318090.py:69: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  securities_df['buy_amount'] = securities_df['buy_amount'].fillna(0)
/tmp/ipython-input-8-1869318090.py:70: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `

Security 39: sell_amount=0.0, holding_gain_loss=0.0, buy_amount=0.0, quantity_yesterday=0.0, close_yesterday=634.55, return=0.0
Security 40: sell_amount=0.0, holding_gain_loss=0.0, buy_amount=0.0, quantity_yesterday=0.0, close_yesterday=0.0, return=0.0
Security 41: sell_amount=0.0, holding_gain_loss=0.0, buy_amount=0.0, quantity_yesterday=0.0, close_yesterday=244.05, return=0.0
Security 42: sell_amount=0.0, holding_gain_loss=0.0, buy_amount=0.0, quantity_yesterday=0.0, close_yesterday=0.0, return=0.0
Security 43: sell_amount=0.0, holding_gain_loss=0.0, buy_amount=0.0, quantity_yesterday=0.0, close_yesterday=0.0, return=0.0
Security 44: sell_amount=0.0, holding_gain_loss=0.0, buy_amount=0.0, quantity_yesterday=0.0, close_yesterday=0.0, return=0.0
Security 45: sell_amount=0.0, holding_gain_loss=0.0, buy_amount=0.0, quantity_yesterday=0.0, close_yesterday=0.0, return=0.0
Security 46: sell_amount=0.0, holding_gain_loss=0.0, buy_amount=0.0, quantity_yesterday=0.0, close_yesterday=0.0, retur

/tmp/ipython-input-8-1869318090.py:68: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  securities_df['buy_quantity'] = securities_df['buy_quantity'].fillna(0)
/tmp/ipython-input-8-1869318090.py:69: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  securities_df['buy_amount'] = securities_df['buy_amount'].fillna(0)
/tmp/ipython-input-8-1869318090.py:70: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `

Security 8: sell_amount=0.0, holding_gain_loss=26740.5, buy_amount=0.0, quantity_yesterday=53481.0, close_yesterday=573.65, return=0.0008716116098666435
Security 9: sell_amount=0.0, holding_gain_loss=-2065397.6000000015, buy_amount=0.0, quantity_yesterday=27034.0, close_yesterday=748.6, return=-0.10205717339032867
Security 10: sell_amount=0.0, holding_gain_loss=-352327.5, buy_amount=0.0, quantity_yesterday=31318.0, close_yesterday=964.1, return=-0.011668914013069184
Security 11: sell_amount=0.0, holding_gain_loss=-600678.5, buy_amount=0.0, quantity_yesterday=18230.0, close_yesterday=1430.2, return=-0.0230387358411411
Security 12: sell_amount=0.0, holding_gain_loss=-98162.25, buy_amount=0.0, quantity_yesterday=23097.0, close_yesterday=874.8, return=-0.004858253315043439
Security 13: sell_amount=0.0, holding_gain_loss=-175335.29999999702, buy_amount=0.0, quantity_yesterday=64939.0, close_yesterday=414.15, return=-0.006519377037305214
Security 14: sell_amount=0.0, holding_gain_loss=-51663

'2023-07-10'